# Gemini Enterprise Data Science Test Drive
Welcome to the Machine Learning part of the Gemini Enterprise Data Science Test Drive.

This jupyter note book will walk you through some exploration of Gemini's ZeroCopy data availability, and how to leverage this function to enable machine learning in Tensorflow

We have pre-configured some data sources as part of this demo that provide time series climate data. Namely, we have measurements of sea ice at the north and south poles, atmospheric carbon measurements from the NOAA observatory at Mauna Loa, and global average and max land temperatures.

This demo will walk through leveraging these data sources, made available via Gemini Enterprise and it's Zero-Copy Data Virtualozation technology, and how they can be used to create some machine learning models using Tensorflow/Keras so that we can make some predictions about the future of climate change. 

## Usage

This is an interactive so-called notebook with multiple cells that contain necessary code (indicated **In [ ]**) and after running the code, output will be shown in a new cell, indicated with **Out [ ]**.
To get started, click the first **In [ ]** cell and press the play icon in the notebook toolar to run the code. Continue running all code cells one by one.
If you prefer to pre-run all code cells, click **Run** -> **Restart Kernel and Run All Cells** from the toolbar

## Step 1: Prepare

Before we begin, we must install jaydebeapi Python package for connecting to zerocopy - testing in progress to include in Jupyter service


In [ ]:
!pip install --user JPype1==0.6.3
!pip install --user jaydebeapi


**Important:** Please restart the kernel after the install below before proceding. Click **Kernel** -> **Restart Kernel and Clear All Outputs** from the toolbar.

Next we import the other pre-requisite packages necessary for our analysis:


In [ ]:
from __future__ import absolute_import, division, print_function
import pathlib
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime as dt
from keras import initializers
from keras.layers import Input, Dense
from keras.models import Model,Sequential
from keras import optimizers
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import r2_score
import jaydebeapi

**Note:** The cell above may produce a warning message which you can ignore.

## Step 2: Establish connection to Zero-Copy Data Virtualization
Using the jaydebeapi Python package, we're going to define a resource that uses a JDBC driver to connect to the Zero-Copy service running in Gemini Enterprise cluster.

In [ ]:
conn = jaydebeapi.connect("org.apache.calcite.avatica.remote.Driver",
"jdbc:avatica:remote:url=http://zero-copyadapter.marathon.l4lb.thisdcos.directory:8765;serialization=JSON",
["admin","admin"],
"/mnt/mesos/sandbox/avatica-1.13.0.jar")

Now that we've configured our connection, let's test out sending it a SQL query so we can see what's returned. CLIMATE.NORTHSEAICE is a table defined in our model.json file as part of our ZeroCopy configuration.

In [ ]:
curs = conn.cursor()
curs.execute("select * from CLIMATE.NORTHSEAICE limit 3")
curs.fetchall()

# Step 3: Explore and prepare data
Great! We know SQL is working so now we can start moving forward with harvesting data from ZeroCopy for analysis. Below is a more complex query that will join several of the ZeroCopy data sources so we have northern and southern sea ice extent, along with land temperatures in a single output:

In [ ]:
climatedata_sql_str = '''select NS."Year", NS."Month", NS."Day", NS."Extent" as "ExtentNorth", SS."Extent" as "ExtentSouth", GT."LandAverageTemperature", GT."LandMaxTemperature"
  FROM CLIMATE.NORTHSEAICE NS
  JOIN CLIMATE.SOUTHSEAICE SS ON NS."Year"=SS."Year" 
      AND NS."Month"=SS."Month"
      AND NS."Day"=SS."Day"
  LEFT JOIN (SELECT SUBSTRING(GT."dt", 1, 4) AS "Year", SUBSTRING(GT."dt", 6, 2) AS "Month", SUBSTRING(GT."dt", 9, 2) AS "Day", 
  "LandAverageTemperature", "LandMaxTemperature" FROM CLIMATE.GLOBALTEMPERATURES GT) GT ON NS."Year"=GT."Year"
      AND NS."Month"=GT."Month"
      AND NS."Day"=GT."Day"'''

With the query configured, we can now send this query to ZeroCopy and read the output straight into a Pandas Dataframe, whhere we can manipulate and massage the data further, if necessary before we start with our modeling tasks:

In [ ]:
climate_sql_query = pd.read_sql_query(climatedata_sql_str, conn)
climate_df = pd.DataFrame(climate_sql_query)
print(climate_df.head(10))

curs = conn.cursor()
curs.execute("select * from CLIMATE.C02MAUNALOA limit 3;")
curs.fetchall()

Unfortunately, our data table has date spread across three columns, one each for day, month, and year. Let's modify our climate_df dataframe to have a single date field. We also need to drop the first row in our climate_df dataframe, which seems to be a header from the original CSV. We should also convery types where necessary, and handle our duplicate columns, as the fields defined as 'NorthExtent' and 'SouthExtent' in the SQL query were not picked up with correct headers here in the dataframe. Let's check our datatypes before we proceed with massaging the data:

In [ ]:
climate_df.dtypes

In [ ]:
climate_df = climate_df.drop([0],axis=0)

In [ ]:
climate_df['Year_Month_Day'] = climate_df['Year']+"-"+climate_df['Month']+"-"+climate_df['Day']
climate_df['Year_Month_Day'] = pd.to_datetime(climate_df['Year_Month_Day'])
climate_df['Extent'] = climate_df['Extent'].astype('float64')
climate_df['LandAverageTemperature'] = climate_df['LandAverageTemperature'].astype('float64')
climate_df['LandMaxTemperature'] = climate_df['LandMaxTemperature'].astype('float64')
climate_df.columns = ['Year','Month','Day','ExtentNorth','ExtentSouth','LandAverageTemperature','LandMaxTemperature','Year_Month_Day']
print(climate_df.head(10))

We'd like to get some atmospheric carbon measurement data as well. Let's set up another query for that and pull it in:

In [ ]:
carbon_sql_str = '''select * from CARBONPPM."carbonppm_maunaloa"'''
carbon_sql_query = pd.read_sql_query(carbon_sql_str, conn)
carbon_df = pd.DataFrame(carbon_sql_query)
print(carbon_df.head(10))

Again we modify the dataframe, to include a date column, convert types where necessary, and remove ' characters and convert negative values to nulls:

In [ ]:
carbon_df['ppm'] = carbon_df["'ppm'"].apply(lambda ppm: ppm.replace("'", ""))
carbon_df['YYYY'] = carbon_df["'YYYY'"].apply(lambda ppm: ppm.replace("'", ""))
carbon_df['M'] = carbon_df["'M'"].apply(lambda ppm: ppm.replace("'", ""))
carbon_df['DD'] = carbon_df["'DD'"].apply(lambda ppm: ppm.replace("'", ""))
carbon_df = carbon_df.drop(["'ppm'","'YYYY'","'M'","'DD'"],axis=1)
carbon_df['Year_Month_Day'] = carbon_df['YYYY']+"-"+carbon_df['M']+"-"+carbon_df['DD']
carbon_df['Year_Month_Day'] = pd.to_datetime(carbon_df['Year_Month_Day'])
carbon_df['ppm'] = carbon_df['ppm'].astype('float64')
carbon_df['ppm'].loc[carbon_df['ppm']==-999.990] = np.nan
carbon_df = carbon_df.drop(['YYYY','M','DD'],axis=1)
print(carbon_df)

Let's merge our two dataframes into a single dataset that we can explore a bit:

In [ ]:
#Join operation to find out same records 
dataset = pd.merge(climate_df,carbon_df,on='Year_Month_Day',how='inner')
#dataset.loc[dataset['ppm']==-999.99] = np.nan
#dataset = dataset[dataset['ppm'] > 0]
dataset

In [ ]:
dataset = dataset.drop(['Year','Month','Day'],axis=1)
dataset.columns = ['ExtentNorth','ExtentSouth','LandAverageTemperature','LandMaxTemperature','Year_Month_Day','PPM']
dataset.dtypes

Plotting is a great way of exploring the dataset. Lets take a look at some plots of northern and southern sea ice extent as a function of atmospheric carbon:

In [ ]:
#Plot the results
dataset.plot(kind='scatter',x='PPM',y='ExtentNorth',color='blue')
plt.show()

In [ ]:
#Plot the results
dataset.plot(kind='scatter',x='PPM',y='ExtentSouth',color='red')
plt.show()

Now let's normalize this data. This means converting all the data to a common scale, without distorting the differences in the range of values. After we normalize all the columns, we will replicate the above plot of northern sea ice as a function of atmospheric carbon. Note how the scales on the axes have changed:

In [ ]:
#Normalizing data set
#dataset.PPM = dataset.PPM.astype(float)
#dataset.ExtentNorth = dataset.ExtentNorth.astype(float)
dataset.PPM = dataset['PPM']/dataset['PPM'].max()
dataset.ExtentNorth = dataset['ExtentNorth']/dataset['ExtentNorth'].max()
dataset.ExtentSouth = dataset['ExtentSouth']/dataset['ExtentSouth'].max()
dataset.LandAverageTemperature = dataset['LandAverageTemperature']/dataset['LandAverageTemperature'].max()
dataset.LandMaxTemperature = dataset['LandMaxTemperature']/dataset['LandMaxTemperature'].max()
plt.plot(dataset.PPM,dataset.ExtentNorth,'g.')

Next lets do a K-means cluster analysis. K-means cluster analysis means seperating the data points into a number of clusters, K, where each data point belongs to the cluster with the nearest mean. In this case K=5. Note we must drop nulls first.

In [ ]:
#Drop nulls
dataset = dataset.dropna()
#K-means cluster analysis on the data set
X = np.array(list(zip(dataset.PPM,dataset.ExtentNorth)))
#print(X)
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.scatter(X[:,0], X[:,1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

Let's also explore our dataset with a few additional plots:

In [ ]:
dataset.plot(kind='scatter',x='PPM',y='ExtentSouth',color='green')
plt.show()

In [ ]:
dataset.plot(kind='scatter',x='PPM',y='LandAverageTemperature',color='blue')
plt.show()

In [ ]:
dataset.plot(kind='scatter',x='PPM',y='LandMaxTemperature',color='black')
plt.show()

Let's return to our original carbon output. Again, we will explore this, plotting atmostpheric carbon as a function of time. Then we'll perform the same normalization operation. Finally, we'll use Tensorflow/Keras to fit a sequential non-linear regression model to the data that we can use to predict future carbon levels by time. We'll do this in a new temporary dataframe to preserve the original:

In [ ]:
carbon_df = carbon_df.sort_values(by=['Year_Month_Day'])
carbon_df = carbon_df[carbon_df.ppm > 0]
carbon_df.plot(kind='line',x='Year_Month_Day',y='ppm',color='green')
plt.show()

In [ ]:
#Normalizing the data set
modified_carbon_data = carbon_df #temp df
modified_carbon_data['Date_Num'] = modified_carbon_data['Year_Month_Day'].map(dt.datetime.toordinal)
modified_carbon_data = modified_carbon_data.drop(['Year_Month_Day'],axis=1)
modified_carbon_data = modified_carbon_data.apply(lambda x:x/x.max(),axis=0)
print(modified_carbon_data)

# Step 4: Build model
Now we can proceed with using Tensorflow/Keras to build our model:

In [ ]:
#Keras sequential model backed by tensorflow
model = Sequential()
model.add(Dense(20, activation='tanh', input_dim=1, kernel_initializer="normal"))
model.add(Dense(1, activation='elu', kernel_initializer="normal"))
# Compile model
model.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
# Fit the model
x = modified_carbon_data['Date_Num']
y = modified_carbon_data['ppm']
model.fit(x, y, epochs=500, batch_size=10,  verbose=2)
predictions = model.predict(x)

Now lets plot the result of this fitting against the original data. Below the model's fit will be in blue against the original data in red:

In [ ]:
#Plot the result
plt.plot(x, y, 'ro', label ='Original data') 
plt.plot(x, predictions, label ='Fitted line') 
plt.title('Non Linear Regression Result') 
plt.legend() 
plt.show()

Next let's return to the sea ice data. Again, we will normalize and plot the data before passing it into Tensorflow. In Tensorflow we will train a sequential model that will attempt to predict future values northern sea ice extent.

In [ ]:
#print(sea_ice['Year_Month_Day'])
climate_df['Date_Num'] = climate_df['Year_Month_Day'].map(dt.datetime.toordinal)

#Data preprocessing
modified_climate_data = climate_df[['Date_Num','ExtentNorth','ExtentSouth','LandAverageTemperature','LandMaxTemperature']] #temp df
modified_climate_data = modified_climate_data.dropna()
modified_climate_data = modified_climate_data.apply(lambda x : x/x.max(),axis=0)
modified_climate_data.plot(kind='line',x='Date_Num',y='ExtentNorth',color='red')


## Step 5: Train model
Now with our second modified dataset, we are again ready to train a model:

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
np.random.seed(7)
#create the data set from previously generated modified climate data
dataframe = pd.concat([modified_climate_data.Date_Num, modified_climate_data.ExtentNorth],axis=1)
dataframe = dataframe['ExtentNorth']
dataset = dataframe.values
dataset = dataset.astype(float)
print(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.70)
print(train_size)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

# reshape into X=t and Y=t+1 for prediction
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

print('Train and Test Data Set')
plt.plot(trainX)
plt.plot(testX)
plt.show()


# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))



# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)



Now we can make some predictions and plot those predictions:

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
#Plot the data set
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back] = trainPredict[:,0]

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1] = testPredict[:,0]
# plot baseline and predictions
plt.plot(dataset,'r')
plt.plot(trainPredictPlot,'g')
plt.plot(testPredictPlot,'b')
plt.show()

## Step 6: Use model to predict value
Now we have trained and fit our model. Let's try and predict the next value for northern sea ice extent.

In [ ]:
#Predict the next value with the model
data_x = [0.33, 0.55, 0.77]
x,y = create_dataset(data_x,look_back) #reform the dataset for next sequence
x_t = np.reshape(x,(x.shape[0],1,x.shape[1]))#reshape the data set to fit into the dl_model
y_t = model.predict(x_t)
print(x_t, ' has the predicted next value: ', y_t)